In [1]:
#importing libraries

In [2]:
import numpy as np
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPool2D

from sklearn.neighbors import NearestNeighbors
import os
from numpy.linalg import norm

In [3]:
#Extract filenames from folder

In [4]:
filenames = []
for file in os.listdir('images'):
    filenames.append(os.path.join('images',file))

In [5]:
len(filenames)

44441

In [6]:
model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
model.trainable = False

model = tf.keras.models.Sequential([model,
                                   GlobalMaxPool2D()
                                   ])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling2d                 │ (None, 2048)                │               0 │
│ (GlobalMaxPooling2D)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [7]:
img = image.load_img('16871.jpg', target_size=(224,224))
img_array = image.img_to_array(img)
img_expand_dim = np.expand_dims(img_array, axis=0)
img_preprocess = preprocess_input(img_expand_dim)
result = model.predict(img_preprocess).flatten()
norm_result = result/norm(result)
norm_result

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


array([0.        , 0.00295298, 0.        , ..., 0.00752506, 0.02113524,
       0.0040174 ], dtype=float32)

In [8]:
def extract_features_from_images(image_path, model):
    img = image.load_img(image_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_expand_dim = np.expand_dims(img_array, axis=0)
    img_preprocess = preprocess_input(img_expand_dim)
    result = model.predict(img_preprocess).flatten()
    norm_result = result/norm(result)
    return norm_result

In [9]:
extract_features_from_images(filenames[0], model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


array([0.        , 0.01761618, 0.00171608, ..., 0.01247239, 0.02726405,
       0.06899218], dtype=float32)

In [ ]:
from tqdm import tqdm

# Batch feature extraction to improve efficiency by processing multiple images at once
def batch_extract_features(filenames, model, batch_size=32):
    image_features = []
    for i in tqdm(range(0, len(filenames), batch_size)):
        batch_files = filenames[i:i+batch_size]
        batch_images = []
        for file in batch_files:
            img = image.load_img(file, target_size=(224,224))
            img_array = image.img_to_array(img)
            batch_images.append(img_array)

        batch_images = preprocess_input(np.array(batch_images))
        batch_features = model.predict(batch_images)
        batch_features = batch_features / np.linalg.norm(batch_features, axis=1, keepdims=True)

        image_features.extend(batch_features)
    return image_features

# Use only the first half of the dataset
half_filenames = filenames[:len(filenames)//2]

image_features = batch_extract_features(half_filenames, model)

  0%|                                                                                          | 0/695 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


  0%|                                                                                  | 1/695 [00:03<46:14,  4.00s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


  0%|▏                                                                                 | 2/695 [00:05<32:06,  2.78s/it]

...

  8%|██████▌                                                                          | 56/695 [02:49<46:56,  4.41s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


  8%|██████▋                                                                          | 57/695 [02:53<45:33,  4.28s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


  8%|██████▊                                                                          | 58/695 [02:58<47:17,  4.46s/it]

In [ ]:
Image_features = pkl.dump(image_features, open('Images_features.pkl','wb'))

In [ ]:
filenames = pkl.dump(filenames, open('filenames.pkl','wb'))

In [ ]:
#Loading Pickle Files

In [ ]:
Image_features = pkl.load(open('Images_features.pkl','rb'))

In [ ]:
filenames = pkl.load(open('filenames.pkl','rb'))

In [ ]:
np.array(Image_features).shape

In [ ]:
neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')

In [ ]:
neighbors.fit(Image_features)

In [ ]:
input_image = extract_features_from_images('16871.jpg',model)

In [ ]:
distance,indices = neighbors.kneighbors([input_image])

In [ ]:
indices[0]

In [ ]:
from IPython.display import Image

In [ ]:
Image('16871.jpg')

In [ ]:
Image(filenames[indices[0][1]])

In [ ]:
Image(filenames[indices[0][2]])

In [ ]:
Image(filenames[indices[0][3]])

In [ ]:
Image(filenames[indices[0][4]])

In [ ]:
Image(filenames[indices[0][5]])